In [1]:
import os
import sys
import tushare as ts
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings.local')
sys.path.append(os.path.join('.', 'mystock'))

import numpy as np
import pandas as pd
from pyecharts import Bar
from datareader import backend
from datareader.echart import *

In [25]:
df =pd.read_html( 'http://quotes.money.163.com/service/gdfx.html?ltdate=2017-09-30%2C2015-06-30&symbol=601600',header=1, encoding='utf-8')[0]

In [38]:
df2 = pd.read_html( 'http://quotes.money.163.com/service/gdfx.html?date=2017-09-30&symbol=601600',header=1, encoding='utf-8')[0]


In [138]:
dfs = []
for q in pd.date_range(start='2007-01-01', end='2018-03-26',  freq='q'):
    d = '%s-%s-%s' %( q.year, q.month, q.day)
    _tmp = pd.read_html( 'http://quotes.money.163.com/service/gdfx.html?date=%s&symbol=601600' % d,header=0, encoding='utf-8')[0]
    _tmp['quarter'] = d
    dfs.append(_tmp)
    
df = pd.concat(dfs,ignore_index=True)
df = df.drop(index=df[df['十大股东']=='十大股东'].index)
df['short_name'] = df['十大股东'].apply(lambda x:x[0:4])

df2 = df[['持有比例','本期持有股(万股)','short_name','quarter']].groupby(['short_name','quarter']).sum()
df2

持有比例         本期持有股(万股)
short_name quarter                                  
JF A       2007-12-31        1.84%          23736.40
JP M       2007-12-31        1.93%          24902.40
           2007-6-30         2.33%             30077
UBS        2007-12-31        1.76%          22697.41
上海浦东       2013-12-31        0.41%           5611.65
           2014-3-31         0.34%           4591.65
上银瑞金       2015-6-30         0.65%           9653.17
           2016-6-30         0.65%           9653.17
           2016-9-30         0.65%           9653.17
中国人寿       2017-6-30         0.33%           4989.34
中国信达       2007-12-31        6.99%          90055.91
           2007-6-30         6.99%           90055.9
           2008-12-31        6.65%          90055.91
           2008-6-30         6.66%           90055.9
           2009-12-31        6.66%           90055.9
           2009-6-30         6.65%          90055.90
           2010-12-31        5.92%          80075.91
           2010-6-30         6.66%          90055.91
           2011-12-31        5.92%          80075.91
           2011-6-30         5.92%          80075.91
           2012-12-31        5.92%          80075.91
           2012-6-30         5.92%          80075.91
           2013-12-31        5.66%          76575.91
           2013-6-30         5.92%          80075.91
           2013-9-30         5.92%          80075.91
           2014-12-31        3.05%          41241.18
           2014-3-31         4.44%          60000.00
           2014-6-30         4.44%          60000.00
           2014-9-30         3.63%          49107.19
           2015-12-31        0.99%          14725.34
...                            ...               ...
香港中央       2007-6-30        23.71%            305591
           2008-12-31       29.06%         393040.98
           2008-6-30        29.12%            393799
           2009-12-31       29.05%            392903
           2009-6-30        29.06%         393055.10
           2010-12-31       29.04%         392690.76
           2010-6-30        29.04%         392782.19
           2011-12-31       29.02%         392510.29
           2011-6-30        29.03%         392611.08
           2012-12-31       29.02%         392427.99
           2012-6-30        29.02%         392435.39
           2013-12-31       29.02%         392492.54
           2013-6-30        29.02%         392495.73
           2013-9-30        29.02%         392417.14
           2014-12-31       29.04%         392794.44
           2014-3-31        29.02%         392488.74
           2014-6-30        29.02%         392458.45
           2014-9-30        29.02%         392542.24
           2015-12-31       26.36%         392895.84
           2015-3-31        29.05%         392825.44
           2015-6-30        26.37%         392943.84
           2015-9-30        26.37%         392945.24
           2016-12-31       26.37%         393007.05
           2016-3-31        26.37%         392960.70
           2016-6-30        26.37%         392946.69
           2016-9-30        26.37%         392984.22
           2017-12-31       26.38%         393229.68
           2017-3-31        26.37%         393029.05
           2017-6-30   26.37%0.39%  393041.255828.17
           2017-9-30        26.38%         393209.91

[304 rows x 2 columns]

In [221]:

from pyecharts import Line
line = Line()

min_date = min(df2.index.levels[1])
max_date = max(df2.index.levels[1])
print(min_date, max_date)
for gd in df2.index.levels[0].unique():
#     print(gd)
#     if gd != '中国证券':
#         continue
    if max(df2.loc[gd]['持有比例'].apply(lambda x : float( x.split('%')[0]))) < 2:
        continue
    tmp = df2.loc[gd]
    tmp.index = pd.DatetimeIndex(tmp.index)
    tmp = tmp.sort_index()
    tmp = tmp.reindex(pd.date_range(min_date, max_date, freq='q'), method = 'bfill')
    
    
    line.add(gd, tmp.index, tmp['本期持有股(万股)'])
    


2007-12-31 2017-9-30


In [222]:
line

In [187]:
gd = '中国证券'

max(df2.loc[gd]['持有比例'].apply(lambda x : float( x.split('%')[0])))

3.15

In [190]:
df2.loc[gd]

,持有比例,本期持有股(万股)
quarter,,
2007-6-30,23.71%,305591
2008-12-31,29.06%,393040.98
2008-6-30,29.12%,393799
2009-12-31,29.05%,392903
2009-6-30,29.06%,393055.10
2010-12-31,29.04%,392690.76
2010-6-30,29.04%,392782.19
2011-12-31,29.02%,392510.29
2011-6-30,29.03%,392611.08


In [ ]:
'a'